##### In order to test distributed code on a "local" cluster

Verify you have Java jdk 8 or higher version

In [1]:
!java -version

java version "1.8.0_251"
Java(TM) SE Runtime Environment (build 1.8.0_251-b08)
Java HotSpot(TM) 64-Bit Server VM (build 25.251-b08, mixed mode)


Check python version

In [2]:
!python --version

Python 3.7.6


Download spark 2.4.5 with hadoop 2.7 at spark.apache.org/downloads.html

Move downloaded file to same folder as this notebook then extract tar

In [ ]:
!tar -xzf spark-2.4.5-bin-hadoop2.7.tgz

* Move extracted folder to C:\spark
* Setup environment variables
    * Windows 10 search environment variables
    * Add:
        * SPARK_HOME = C:\spark\spark-2.4.5-bin-hadoop2.7
        * HADOOP_HOME = C:\spark\spark-2.4.5-bin-hadoop2.7
    * Add to PATH:
        * C:\spark\spark-2.4.5-bin-hadoop2.7\bin

#### Download and setup winutils.exe

Go to  https://github.com/steveloughran/winutils , choose hadoop version, goto bin and download winutils.exe
* Then move your winutils.exe in to bin directory of spark installation (C:\spark\spark-2.4.5-bin-hadoop2.7.tgz\bin)

#### To ensure functionality

* Create folder C:\tmp\hive
* Open command promtp (cmd) with "run as administrator" then execute:
    * winutils.exe chmod -R 777 C:\tmp\hive
    * winutils.exe ls -F C:\tmp\hive
* output should look something like: drwxrwxrwx|1|LAPTOP-.....

#### Now install to jupyter notebook

Open another anaconda shell and execute:
* conda install -c conda-forge findspark

### Now initialize PySpark!!

In [3]:
import findspark
findspark.init()

findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-2.4.5-bin-hadoop2.7'

##### Basic "Hello world" style program for testing PySpark

In [4]:
import pyspark
sc = pyspark.SparkContext('local[*]')

txt = sc.textFile('states.txt')
print(txt.count())

python_lines = txt.filter(lambda line: 'python' in line.lower())
print(python_lines.count())

51
0


In [5]:
sc.stop()

## Project Code below!

import statements and bring data into RDD

In [11]:
from pyspark import *
from pyspark.sql import *

In [52]:
from pyspark.sql.types import StructType, StructField, LongType

def with_column_index(sdf): 
    new_schema = StructType(sdf.schema.fields + [StructField("ColumnIndex", LongType(), False),])
    return sdf.rdd.zipWithIndex().map(lambda row: row[0] + (row[1],)).toDF(schema=new_schema)

In [25]:
spark = SparkSession.builder.appName("COVID-19 GCN").config("").getOrCreate()
#sc = pyspark.SparkContext('local[*]')

eg = spark.read.option("header", "false").option("delimiter", " ").csv('edge_list/edge_index.txt')
eg.show()

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|  0|  9| {}|
|  0| 10| {}|
|  0| 42| {}|
|  0| 24| {}|
|  9|  1| {}|
|  9| 10| {}|
| 10| 40| {}|
| 10| 33| {}|
| 10| 42| {}|
| 42|  3| {}|
| 42| 17| {}|
| 42| 24| {}|
| 42| 25| {}|
| 42| 33| {}|
| 42| 46| {}|
| 24|  3| {}|
| 24| 18| {}|
|  1|  4| {}|
|  1| 47| {}|
|  1| 43| {}|
+---+---+---+
only showing top 20 rows



In [85]:
nodeProps = spark.read.options(header='false', inferschema='true', delimiter=' ').csv('./datesprepped/2020-01-31.csv')
#nodeProps = nodeProps.withColumnRenamed("_c0","Search Terms")
#nodeProps = nodeProps.withColumn("index", row_number().over("Search Terms"))
#nodeProps = with_column_index(nodeProps)
nodeProps.show()

+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----

In [55]:
hopkinData = spark.read.options(header='false', inferschema='true').csv('./hopkins_data/cleaned/2020-01-31.csv')
hopkinData = hopkinData.withColumnRenamed("_c0","Confirmed").withColumnRenamed("_c1","Deaths").withColumnRenamed("_c2","Recovered")
hopkinData = with_column_index(hopkinData)
hopkinData.show()

+---------+------+---------+-----------+
|Confirmed|Deaths|Recovered|ColumnIndex|
+---------+------+---------+-----------+
|        0|     0|        0|          0|
|        0|     0|        0|          1|
|        1|     0|        0|          2|
|        0|     0|        0|          3|
|        2|     0|        0|          4|
|        0|     0|        0|          5|
|        0|     0|        0|          6|
|        0|     0|        0|          7|
|        0|     0|        0|          8|
|        0|     0|        0|          9|
|        0|     0|        0|         10|
|        0|     0|        0|         11|
|        0|     0|        0|         12|
|        2|     0|        0|         13|
|        0|     0|        0|         14|
|        0|     0|        0|         15|
|        0|     0|        0|         16|
|        0|     0|        0|         17|
|        0|     0|        0|         18|
|        0|     0|        0|         19|
+---------+------+---------+-----------+
only showing top

In [76]:
stateProps = nodeProps.join(hopkinData, nodeProps.ColumnIndex == hopkinData.ColumnIndex, 'inner').drop("ColumnIndex")
#stateProps.orderBy('Confirmed', ascending=False)
stateProps = with_column_index(stateProps)
stateProps.show()

+--------------------+---------+------+---------+-----------+
|        Search Terms|Confirmed|Deaths|Recovered|ColumnIndex|
+--------------------+---------+------+---------+-----------+
|0 100 0 0 0 49 0 ...|        0|     0|        0|          0|
|0 0 0 0 0 30 0 0 ...|        0|     0|        0|          1|
|0 69 0 0 0 33 0 0...|        0|     0|        0|          2|
|0 0 0 0 0 22 0 0 ...|        0|     0|        0|          3|
|0 0 0 0 0 4 0 0 0...|        0|     0|        0|          4|
|0 0 0 0 0 0 0 0 0...|        0|     0|        0|          5|
|0 0 0 0 0 61 0 0 ...|        0|     0|        0|          6|
|0 0 0 0 0 0 0 0 0...|        0|     0|        0|          7|
|0 3 0 0 0 16 0 0 ...|        0|     0|        0|          8|
|0 12 0 0 0 10 0 0...|        0|     0|        0|          9|
|0 0 0 0 0 42 0 0 ...|        0|     0|        0|         10|
|0 0 0 0 0 0 0 0 0...|        0|     0|        0|         11|
|0 0 0 0 0 10 0 0 ...|        0|     0|        0|         12|
|0 20 0 

#### Convert dataframe to RDD then to LabeledPoint say it can be saved in the LibSVM format to be used with sparks MLLib api.

In [82]:
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

In [81]:
c = stateProps.rdd

In [83]:
d = c.map(lambda line: LabeledPoint(line[0],[line[1:]]))

In [84]:
MLUtils.saveAsLibSVMFile(d, "/data/")

Py4JJavaError: An error occurred while calling o1089.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:385)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDD

In [80]:
stateProps.write.format("libsvm").save("statePropSVM")

Py4JJavaError: An error occurred while calling o1041.save.
: java.io.IOException: Illegal schema for libsvm data, schema=StructType(StructField(Search Terms,StringType,true), StructField(Confirmed,IntegerType,true), StructField(Deaths,IntegerType,true), StructField(Recovered,IntegerType,true), StructField(ColumnIndex,LongType,false))
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat.verifySchema(LibSVMRelation.scala:86)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat.prepareWrite(LibSVMRelation.scala:122)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:103)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql

In [70]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [77]:
labelIndexer = StringIndexer(inputCol="ColumnIndex", outputCol="indexedLabel").fit(stateProps)

In [79]:
featureIndexer = VectorIndexer(inputCol="Confirmed", outputCol="indexedTerms", maxCategories=4).fit(stateProps)

IllegalArgumentException: 'requirement failed: Column Confirmed must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually int.'